In [ ]:
import numpy as np
from collections import defaultdict
import json

In [ ]:
# all this +/- stuff is only really used for the 'current year MOV' prediction
iv = np.array([72.7 ,  7.28, 18.1 ])

def get_mov(x):
    offset = len(iv)//3
    tot = -2.021381747657189
    for i in range(offset):
        tot += (iv[offset+i])*(np.tanh((x-iv[i])/iv[2*offset+i])+1.0)
    return tot

# estimated mov per ovr
o2m = {}
for i in np.arange(101):
    o2m[i] = get_mov(i)

# replacement level
r_lvl = -1.4351774267651591

age_shift ={19: 6.6,
 20: 7.2,
 21: 5.1,
 22: 4.6,
 23: 2.2,
 24: 2.2,
 25: 0.2,
 26: 0.1,
 27: -1.0,
 28: -1.1,
 29: -1.7,
 30: -2.4,
 31: -3.4,
 32: -3.5,
 33: -3.6,
 34: -4.7,
 35: -5.0}

max_shift = min(age_shift.values())

# replacement level
RL = 44.37

# expected mov, weight for players, weight for salary cap space
weights = {
0 : ( [-1.019, 0.99] , 1.22 ),
1 : ( [9.559, 0.631] , 2.05 ),
2 : ( [7.214, 0.37] , 3.62 ),
}

# draft value
draftP = np.array([0.27988742, 0.30226007, 0.62866095])
def winp_draft(ovr,pot,age):
    xv = 4.3341 + ovr*0.1294 + pot*0.0343 + age*(-0.7099)
    return 1/(1+np.exp(-xv))


# team value
team_mov = [ -0.20384938,   0.3719406,  101.37586688]

# salary multiplier
sA = 4.020403849764475

In [ ]:
age_shift_int = defaultdict(int)
age_min,age_max = min(age_shift.keys()),max(age_shift.keys())
total_age_prog = sum([v for k,v in age_shift.items() if v > 0])
for age in range(age_min,age_max):
    left_over = sum([age_shift[_] for _ in range(age,age_max) if age_shift[_] >=0])
    if left_over > 0:
        age_shift_int[age] = left_over/total_age_prog
age_shift_int    

In [ ]:
team = { 'p' : [(22,66,22000,2,0),(22,66,22000,2,0),(22,66,22000,2,0),(22,66,22000,5,0)], #(age,ovr,salary,years_left,draf_val)
         'd' : [(1, 0, 8.6),(1, 0, -5.35),(2,0,3),(1,1,3),(2,1,3),(1,2,3),(2,2,3),(1,3,3),(2,3,3),(1,4,3),(2,4,3)] #(round,years_left,team_MOV)
       }
def get_team_value(team,sCap=90000,minS=750):
    YEARS_TO_MODEL = 3
    teamNum = 30
    
    # turn mov into draft pick and future mov
    m2pos = lambda x: int(round(np.clip( (teamNum-1)/(1+np.exp(0.0048 - 0.4037*(x))),0,teamNum-1)))
    m2next = lambda year,mov: [1,0.5,0.25,0.08,0.03,0.01,0.01,0.01,0.01,0.01][year]*mov 

    # turn draft picks into specific predictions
    draft_picks = [ (yr,m2pos(m2next(yr,mov))+teamNum*(rnd-1)) for rnd,yr,mov in team['d']]
    
    pars = defaultdict(list)
    tss = defaultdict(int)
    dpars = []

    dpars = []
    for i in range(YEARS_TO_MODEL):
        for age,ovr,con,yrl,draft_v in team['p']:
            tss[i] += con
            ovr2 = ovr
            for j in range(max(0,i-1)):
                ovr2+=age_shift.get(age+j,max_shift)
            pars[i].append(ovr2)
    for age,ovr,con,yrl,draft_v in team['p']:
        if yrl >= YEARS_TO_MODEL:
            #print('yES',(age,ovr,con,yrl))
            ovr2  =ovr
            povrs = [ovr2]
            for j in range(yrl-1):
                ovr2+=age_shift.get(age+j,max_shift)
                povrs.append(ovr2)
            pmovs = [o2m[int(np.clip(np.round(ovr2),0,100))] for ovr2 in povrs]
            ccont = [(sCap/3)*(pmov-r_lvl)/sA for pmov in pmovs]
            cvals = [c-con for c in ccont]
            amount_to_add = sum(cvals[YEARS_TO_MODEL:])/YEARS_TO_MODEL
            for i  in range(YEARS_TO_MODEL):
                tss[i] -= amount_to_add
    
    for yr,p in draft_picks:
        dpars.append((0.95**yr)*draftP[1]*np.exp(-draftP[0]*p**draftP[2]))
        
    for age,ovr,con,yrl,draft_v in team['p']:
        dpars.append(age_shift_int[age]*draft_v)
    
    pred_movs = []
    for i in range(YEARS_TO_MODEL):
        play = [p for p in pars[i] if p >= RL]
        lp = len(play)
        if lp < 10:
            play= play + (10-lp)*[RL]
        play = sorted(play,reverse=True)[:10]
        play_s = sum([np.exp(i*team_mov[0])*p for i,p in enumerate(play)])*team_mov[1] -team_mov[2]

        cap_hit = tss[i] + (10-lp)*minS 

        diff = (sCap-cap_hit)/sCap
        cap_space = np.maximum(diff,0.1*diff)
        x = weights.get(i,weights[2])

        p_mov = x[1] * (x[0][0] * cap_space + x[0][1] * play_s)
        
        pred_movs.append(p_mov)
        
    cA,cB = (.47854580217902276, -4.016030120527145)

    win_p = [1.0/(1+np.exp(-mov*cA -cB)) for mov in pred_movs]

    # discount factor for the future, more uncertainty, less sure reward
    value = [wp*(0.95**(i)) for i,wp in enumerate(win_p)]
    #print(sum(value),sum(dpars))
    return sum(value) + sum(dpars)
get_team_value(team,90000,750)

In [ ]:
data = json.load(open('real_2020.json','rt',encoding='utf-8-sig'))
gA = {_['key']:_['value'] for _ in data['gameAttributes']}

In [ ]:
team_players = defaultdict(list)
team_names = {}
season = gA['season']
sCap = gA['salaryCap']
minS = gA['minContract']
maxS = gA['maxContract']

team_injuries = defaultdict(list)
for p in data['players']:
    if p['tid'] >= 0:
        ovr = p['ratings'][-1]['ovr']
        age = season - p['born']['year']
        salary = p['contract']['amount']
        years_left = p['contract']['exp']-season
        dp = p['draft']['pick'] + 30 * (p['draft']['round']-1)
        if dp < 0:
            dp = 61
        team_players[p['tid']].append((age,ovr,salary,years_left,draftP[1]*np.exp(-draftP[0]*dp**draftP[2])))
        team_injuries[p['tid']].append(p['injury']['gamesRemaining'])

In [ ]:
team_movs = defaultdict(float)

for t in data['teams']:
    tid = t['tid']
    team_names[t['tid']] = t['abbrev']
    for ts in t['stats']:
        if ts['playoffs']:
            continue
        current_mov = 0
        gp = ts['gp']+1e-9
        gl = 82-gp+1e-9
        if season == ts['season'] and not ts['playoffs'] and ts['gp']>0:
            mov = (ts['pts'] - ts['oppPts']) / ts['gp'];
            current_mov = mov
        estimated_mov = sum(sorted([o2m[_[1]]*(max(gl-i,0)/gl) for _,i in zip(team_players[tid],team_injuries[tid])])[-10:])
        team_movs[tid] = (gp/82)*current_mov + (gl/82)*estimated_mov    

In [ ]:
team_picks = defaultdict(list)
for d in data['draftPicks']:
    mov = team_movs[d['originalTid']]
    tid = d['tid']
    rnd = d['round']
    yl = d['season']-season
    team_picks[tid].append((rnd,yl,mov))

In [ ]:
teams_vals = []
for i in range(len(data['teams'])):
    #print(team_names[i])
    val = get_team_value({'mov':team_movs[i],'p':team_players[i],'d':team_picks[i]},sCap,minS)
    teams_vals.append((val,team_names[i]))

In [ ]:
for v,t in sorted(teams_vals)[::-1]:
    print(round(100*v,2),t)

In [ ]:
import random
deals = []
for i in range(1000):
    t1 = np.random.randint(30)
    t2 = np.random.randint(30)
    if t1 == t2:
        continue
    
    t1vo = get_team_value({'mov':team_movs[t1], 'p':team_players[t1],'d':team_picks[t1]},sCap,minS)
    t2vo = get_team_value({'mov':team_movs[t2], 'p':team_players[t2],'d':team_picks[t2]},sCap,minS)
    pn = np.random.randint(len(team_picks[t1]))
    picks = [_ for _ in team_picks[t1]]
    pick = picks[pn]        
    del picks[pn]

    for pi in range(len(team_players[t2])):
        local_p = [_ for _ in team_players[t2]]
        player = local_p[pi]
        del local_p[pi]
        t1v = get_team_value({'mov':team_movs[t1],'p':team_players[t1] + [player],'d':picks},sCap,minS)
        t2v = get_team_value({'mov':team_movs[t2],'p':local_p,'d':team_picks[t2] + [pick]},sCap,minS)
        if t1v > t1vo and t2v > t2vo:
            val = min((t1v-t1vo),(t2v-t2vo))
            deals.append((val,team_names[t1],pick,team_names[t2],player))

In [ ]:
sorted(deals,reverse=True)

In [ ]:
import random
deals = []
for i in range(1000):
    t1 = np.random.randint(30)
    t2 = np.random.randint(30)
    if t1 == t2:
        continue
    
    t1vo = get_team_value({'mov':team_movs[t1],'p':team_players[t1],'d':team_picks[t1]},sCap,minS)
    t2vo = get_team_value({'mov':team_movs[t2],'p':team_players[t2],'d':team_picks[t2]},sCap,minS)
    
    pn = np.random.randint(len(team_players[t1]))
    players1 = [_ for _ in team_players[t1]]
    pick = players1[pn]        
    del players1[pn]

    for pi in range(len(team_players[t2])):
        local_p = [_ for _ in team_players[t2]]
        player = local_p[pi]
        del local_p[pi]
        t1v = get_team_value({'mov':team_movs[t1],'p':players1 + [player],'d':team_picks[t1]},sCap,minS)
        t2v = get_team_value({'mov':team_movs[t2],'p':local_p + [pick],'d':team_picks[t2]},sCap,minS)
        if t1v > t1vo and t2v > t2vo:
            val = (t1v-t1vo) + (t2v-t2vo)
            deals.append((val,team_names[t1],pick,team_names[t2],player))

In [ ]:
for d in sorted(deals,reverse=True):
    if d[4][2]/d[2][2] < 1.25 and d[4][2]/d[2][2] > 1/1.25:
        print(d)

In [ ]:
import random
deals = set()
for i in range(1000):
    t1 = np.random.randint(30)
    t2 = np.random.randint(30)
    if t1 == t2:
        continue
    
    t1vo = get_team_value({'mov':team_movs[t1],'p':team_players[t1],'d':team_picks[t1]},sCap,minS)
    t2vo = get_team_value({'mov':team_movs[t2],'p':team_players[t2],'d':team_picks[t2]},sCap,minS)
    
    pn = np.random.randint(len(team_players[t1]))
    players1 = [_ for _ in team_players[t1]]
    pick = players1[pn]        
    del players1[pn]

    for pi in range(len(team_players[t2])):
        local_p = [_ for _ in team_players[t2]]
        player = local_p[pi]
        del local_p[pi]
        if not (pick[2]/player[2] < 1.25 and pick[2]/player[2] > 1/1.25):
            continue
        for p2 in range(len(team_picks[t2])):
            local_picks = [_ for _ in team_picks[t2]]
            pick2 = local_picks[p2]       
            del local_picks[p2]
            t1v1 = get_team_value({'mov':team_movs[t1],'p':players1 + [player],'d':team_picks[t1]},sCap,minS)

            t1v = get_team_value({'mov':team_movs[t1],'p':players1 + [player],'d':team_picks[t1] + [pick2]},sCap,minS)
            t2v = get_team_value({'mov':team_movs[t2],'p':local_p + [pick],'d':local_picks },sCap,minS)
            if t1v1 < t1vo and t1v > t1vo and t2v > t2vo:
                v1 = (t1v-t1vo)
                v2 = (t2v-t2vo)
                val = min( v1 , v2)
                deals.add((val,v1,v2,team_names[t1],pick,team_names[t2],player,pick2))
                

In [ ]:
sorted(list(deals),reverse=True)

In [ ]:
team_names

In [ ]:
team_test = {'mov':team_movs[7],'d':team_picks[7],'p':team_players[7]}

In [ ]:
get_team_value(team_test,sCap,minS)

In [ ]:
team_test2 = {'mov': -5.482382353981168,
 'd': [(1, 0, -5.482382353981168),
  (2, 0, -5.482382353981168),
  (2, 0, 9.085203576451224),
  (1, 1, -5.482382353981168),
  (2, 1, -5.076082713949978),
  (2, 1, 4.0382054499133755),
  (1, 2, -5.482382353981168),
  (2, 2, -5.482382353981168),
  (2, 2, -5.076082713949978),
  (1, 3, -5.482382353981168),
  (2, 3, -5.482382353981168),
  (2, 3, -7.647603332074514),
  (1, 4, -5.482382353981168),
  (2, 4, -5.482382353981168),
  (1, 5, -5.482382353981168),
  (2, 5, -5.482382353981168),
  (2, 5, -0.6651492516752459),
  (1, 6, -5.482382353981168),
  (2, 6, -5.482382353981168),
  (2, 6, -5.051141623873145)],
 'p': [(31, 50, 13350, 2, 0.007400055732949782),
  (26, 58, 17500, 3, 0.036373312235063106),
  (32, 43, 19000, 0, 0.05087529943560001),
  (43, 36, 2500, 0, 0.13997323937839157),
  (25, 27, 1700, 1, 0.007400055732949782),
  (22, 31, 1550, 2, 0.023156748165624817),
  (23, 25, 950, 1, 0.007400055732949782),
  (21, 51, 4850, 3, 0.09192940831774696),
  (22, 56, 4250, 2, 0.05087529943560001),
  (22, 58, 6650, 2, 0.13997323937839157),
  (23, 52, 8050, 3, 0.15481216217313457),
  (23, 59, 4150, 1, 0.05087529943560001),
  (27, 39, 1500, 0, 0.007400055732949782),
  (26, 43, 2600, 0, 0.04531599594096907),
  (24, 47, 2350, 0, 0.031108243072596267),
  (25, 42, 1600, 0, 0.02812560393241915)]}
get_team_value(team_test2,sCap,minS)

In [ ]:
team_test2 = {'mov': -5.482382353981168,
 'd': [
  (2, 0, -5.482382353981168),
  (2, 0, 9.085203576451224),
  (1, 1, -5.482382353981168),
  (2, 1, -5.076082713949978),
  (2, 1, 4.0382054499133755),
  (1, 2, -5.482382353981168),
  (2, 2, -5.482382353981168),
  (2, 2, -5.076082713949978),
  (1, 3, -5.482382353981168),
  (2, 3, -5.482382353981168),
  (2, 3, -7.647603332074514),
  (1, 4, -5.482382353981168),
  (2, 4, -5.482382353981168),
  (1, 5, -5.482382353981168),
  (2, 5, -5.482382353981168),
  (2, 5, -0.6651492516752459),
  (1, 6, -5.482382353981168),
  (2, 6, -5.482382353981168),
  (2, 6, -5.051141623873145)],
 'p': [(31, 50, 13350, 2, 0.007400055732949782),
  (26, 58, 17500, 3, 0.036373312235063106),
  (32, 43, 19000, 0, 0.05087529943560001),
  (43, 36, 2500, 0, 0.13997323937839157),
  (25, 27, 1700, 1, 0.007400055732949782),
  (22, 31, 1550, 2, 0.023156748165624817),
  (23, 25, 950, 1, 0.007400055732949782),
  (21, 51, 4850, 3, 0.09192940831774696),
  (22, 56, 4250, 2, 0.05087529943560001),
  (22, 58, 6650, 2, 0.13997323937839157),
  (23, 52, 8050, 3, 0.15481216217313457),
  (23, 59, 4150, 1, 0.05087529943560001),
  (27, 39, 1500, 0, 0.007400055732949782),
  (26, 43, 2600, 0, 0.04531599594096907),
  (24, 47, 2350, 0, 0.031108243072596267),
  (25, 42, 1600, 0, 0.02812560393241915)]}
get_team_value(team_test2,sCap,minS)

In [ ]:
team_test = {'mov':team_movs[2],'d':team_picks[2],'p':team_players[2]}
get_team_value(team_test,sCap,minS)

In [ ]:
team_test2 = {'mov': -5.051141623873145,
 'd': [(2, 0, 1.5294398686913402),
  (2, 0, -0.11763123883654236),
  (2, 1, 4.253735265080072),
  (1, 1, -5.051141623873145),
  (1, 1, -3.4271516708022802),
  (2, 1, -3.4271516708022802),
  (1, 2, -5.051141623873145),
  (2, 2, -5.051141623873145),
  (2, 2, 6.640651174426148),
  (1, 3, -5.051141623873145),
  (1, 5, -5.051141623873145),
  (2, 5, -5.051141623873145),
  (1, 6, -5.051141623873145)],
 'p': [(24, 51, 4800, 2, 0.02812560393241915),
  (24, 55, 1650, 2, 0.01680073169357861),
  (25, 59, 29550, 3, 0.22846896618569987),
  (27, 22, 1650, 2, 0.007400055732949782),
  (21, 25, 3900, 3, 0.031108243072596267),
  (30, 59, 20200, 4, 0.022088150606323197),
  (30, 64, 38000, 4, 0.08541509934145018),
  (32, 68, 40250, 2, 0.11675830647455769),
  (26, 26, 1650, 2, 0.007400055732949782),
  (23, 46, 1800, 1, 0.10742693347018893),
  (27, 26, 100, 0, 0.007400055732949782),
  (20, 33, 1750, 3, 0.018372688753582653),
  (23, 38, 200, 0, 0.15481216217313457),
  (28, 38, 1850, 2, 0.007400055732949782),
  (23, 29, 1650, 2, 0.007400055732949782)]}
get_team_value(team_test2,sCap,minS)

In [ ]:
team_test = {'mov':team_movs[22],'d':team_picks[22],'p':team_players[22]}

In [ ]:
team_test = {'mov': 1.5294398686913402,
 'd': [(1, 0, 1.5294398686913402),
  (2, 0, -5.051141623873145),
  (2, 1, 1.5294398686913402),
  (1, 2, 1.5294398686913402),
  (2, 2, 1.5294398686913402),
  (2, 3, 1.5294398686913402),
  (2, 3, 4.0382054499133755),
  (1, 4, 1.5294398686913402),
  (2, 4, 1.5294398686913402),
  (1, 5, 1.5294398686913402),
  (2, 5, 1.5294398686913402),
  (1, 6, 1.5294398686913402),
  (2, 6, 1.5294398686913402)],
 'p': [(32, 48, 5600, 1, 0.007400055732949782),
  (23, 26, 900, 1, 0.007400055732949782),
  (29, 52, 10050, 3, 0.014115227391274273),
  (27, 46, 2250, 1, 0.12747982320989207),
  (25, 66, 31650, 4, 0.15481216217313457),
  (27, 44, 13000, 0, 0.19608449676058912),
  (28, 54, 8500, 3, 0.007400055732949782),
  (21, 66, 8100, 2, 0.17293338922302573),
  (26, 26, 900, 1, 0.007400055732949782),
  (35, 38, 12000, 0, 0.042834501080659414),
  (36, 41, 3650, 0, 0.007400055732949782),
  (28, 56, 17750, 1, 0.038378007300704536),
  (28, 36, 1900, 0, 0.09921659340181611),
  (30, 48, 8000, 3, 0.007400055732949782),
  (24, 44, 1550, 2, 0.024289644129968053),
  (28, 55, 9000, 2, 0.047989461040794185),
  (25, 48, 5050, 1, 0.06506895926938651),
  (27, 58, 4000, 2, 0.007400055732949782)]}
get_team_value(team_test,sCap,minS)

In [ ]:
team_test = {'mov': 1.5294398686913402,
 'd': [(1, 0, 1.5294398686913402),
  (2, 0, -5.051141623873145),
  (2, 1, 1.5294398686913402),
  (1, 2, 1.5294398686913402),
  (2, 2, 1.5294398686913402),
  (2, 3, 1.5294398686913402),
  (2, 3, 4.0382054499133755),
  (1, 4, 1.5294398686913402),
  (2, 4, 1.5294398686913402),
  (1, 5, 1.5294398686913402),
  (2, 5, 1.5294398686913402),
  (1, 6, 1.5294398686913402),
  (2, 6, 1.5294398686913402)],
 'p': [(32, 48, 5600, 1, 0.007400055732949782),
  (23, 26, 900, 1, 0.007400055732949782),
  (29, 52, 10050, 3, 0.014115227391274273),
  (27, 46, 2250, 1, 0.12747982320989207),
  (25, 66, 31650, 4, 0.15481216217313457),
  (27, 44, 13000, 0, 0.19608449676058912),
  (28, 54, 8500, 3, 0.007400055732949782),
  (21, 66, 8100, 2, 0.17293338922302573),
  (26, 26, 900, 1, 0.007400055732949782),
  (35, 38, 12000, 0, 0.042834501080659414),
  (36, 41, 3650, 0, 0.007400055732949782),
  (28, 56, 17750, 1, 0.038378007300704536),
  (28, 36, 1900, 0, 0.09921659340181611),
  (30, 48, 8000, 3, 0.007400055732949782),
  (24, 44, 1550, 2, 0.024289644129968053),
  (28, 55, 9000, 2, 0.047989461040794185),
  (25, 48, 5050, 1, 0.06506895926938651),
  (27, 58, 4000, 2, 0.007400055732949782)]}
get_team_value(team_test,sCap,minS)